In [1]:
!pip install python-dotenv
!git init
!touch .gitignore
!echo "info.env" >> .gitignore
!git check-ignore -v .env


hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/.git/


In [13]:
import pandas as pd
import http.client
import json
import requests
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv("info.env")

# Access credentials
authUsername = os.getenv("API_USERNAME")
authPassword = os.getenv("API_PASSWORD")

#define URL for the API request
url = 'https://api.ravelry.com/color_families.json'
#make the request
r = requests.get(url, auth=requests.auth.HTTPBasicAuth(authUsername, authPassword))
#close the connection
r.close()

#print("response code: {}".format(r)) #tells me the response code
#print("response code Details: {}".format(r.iter_lines())) #tells me details about the response code
#print("response output:")
#print(r.text) #tells me the output
#print("response output formatted:")

query = 'bear'
page = 9
page_size = 10
craft = 'crochet'
#define URL for the API request
url = 'https://api.ravelry.com/patterns/search.json?query={}&page={}&page_size={}&craft={}'.format(query, page, page_size, craft)
#make the request
r = requests.get(url, auth=requests.auth.HTTPBasicAuth(authUsername, authPassword))
#close the connection
r.close()

yarndf = pd.DataFrame(columns = ['crochet_gauge', 'id', 'knit_gauge',
                                 'max_gauge', 'min_gauge', 'name',
                                 'ply', 'wpi', 'yardage'])

if r.status_code == 200:
    data = r.json()
    # Extract pattern ID(s) from the search results
    if 'patterns' in data and len(data['patterns']) > 0:
      for i in range(len(data['patterns'])):
          pattern_id = data['patterns'][i]['id']

          # Define URL to get pattern details
          pattern_url = f'https://api.ravelry.com/patterns/{pattern_id}.json'

          # Make the request for pattern details
          pattern_response = requests.get(pattern_url, auth=requests.auth.HTTPBasicAuth(authUsername, authPassword))
          pattern_response.close()

          # If patterns found, collect the necessary yarn info
          if pattern_response.status_code == 200:
              pattern_data = pattern_response.json()
              yarnData = pattern_data['pattern']['yarn_weight']
              yarnData['yardage'] = pattern_data['pattern']['yardage_description']
              yarndf = pd.concat([yarndf, pd.DataFrame([yarnData])], ignore_index=True)

    else:
      print("No patterns found")
else:
  print("Unable to access patterns" + r.code())

print(yarndf)


  crochet_gauge  id knit_gauge max_gauge min_gauge         name   ply   wpi  \
0          None  12         20      None      None      Worsted    10     9   
1                 1         18      None      None         Aran    10     8   
2                 1         18      None      None         Aran    10     8   
3          None  12         20      None      None      Worsted    10     9   
4                 6       7-12      None      None  Super Bulky  None   5-6   
5          None  15       None      None      None    Any gauge  None  None   
6                 1         18      None      None         Aran    10     8   
7          None  11         22      None      None           DK     8    11   
8          None  11         22      None      None           DK     8    11   
9          None  12         20      None      None      Worsted    10     9   

             yardage  
0              yards  
1              yards  
2              yards  
3              yards  
4  1400 - 1450 